__<font color='red'>
Note: Visualizations have been set to False to keep the notebook size small. When running the notebook locally set the visualize variable to True to display the plots.
</font>__

## Setup and loading traces
In this demo we use 8 trace files. To run this demo notebook on your laptop
1. Clone the repo `git clone https://github.com/facebookresearch/HolisticTraceAnalysis.git`
1. [Optional and recommended] Setup a conda environment. See README for details.
1. Set the `trace_dir` parameter in the next cell to the location of `vision_transformer` folder in your local `HolisticTraceAnalysis` installation.

In [ ]:
from hta.trace_analysis import TraceAnalysis

hta_dir = "<PATH TO HTA>"
trace_dir = hta_dir + "/tests/data/vision_transformer/"
analyzer = TraceAnalysis(trace_dir=trace_dir)

# Temporal Breakdown

### Description
This feature gives a breakdown of time spent by the GPU as follows:
1. Idle time - GPU idle
1. Compute time	- GPU busy with computation events
1. Non compute time - GPU busy with communication or memory events

### API

In [2]:
analyzer.get_temporal_breakdown?

### Example

In [3]:
time_spent_df = analyzer.get_temporal_breakdown(visualize=False)

In [4]:
time_spent_df

,rank,idle_time(ns),compute_time(ns),non_compute_time(ns),kernel_time(ns),idle_time_pctg,compute_time_pctg,non_compute_time_pctg
0,0,552069,596651,884850,2033570,27.15,29.34,43.51
1,1,431771,596759,1004227,2032757,21.24,29.36,49.40
2,2,312107,596886,1124788,2033781,15.35,29.35,55.31
3,3,274646,604137,1154491,2033274,13.51,29.71,56.78
4,4,418833,598040,1021824,2038697,20.54,29.33,50.12
5,5,318972,601581,1112561,2033114,15.69,29.59,54.72
6,6,388040,598029,1047787,2033856,19.08,29.40,51.52
7,7,454830,599358,979022,2033210,22.37,29.48,48.15


# Kernel Breakdown

### Description

This feature computes the following:
1. Breakdown of time spent among kernel types (Computation, Communication, Memory) across all ranks.
1. Kernels taking the most time on each rank by kernel type.
1. Distribution of average time across ranks for the kernels taking the most time.

### API

In [5]:
analyzer.get_gpu_kernel_breakdown?

Note:
1. If visualize is True, then
    1. Pie chart of percentage time spent by kernel type.
    1. Pie charts of the most time consuming kernels for each kernel type, for each rank.
    1. Bar graphs for the most time consuming kernels for each kernel type for each rank. The error
        bars indicate the min and max.

1. The number of kernels to display can be tuned using the num_kernels parameter. Additionally, duration_ratio can be used to tune the percentage of time that needs to be analyzed. If both num_kernels and duration_ratio are specified, then num_kernels takes precedence.

1. If include_memory_kernels is set to False then the calculation is done using Communication and Computation kernels only. Mathematically, the intersection calculates the following ratio: (time spent computing while communicating) / Union(time spent communicating, time spent computing)

### Example

In [4]:
kernel_type_metrics_df, kernel_metrics_df = analyzer.get_gpu_kernel_breakdown(visualize = False, 
                                                                              duration_ratio = 0.8,
                                                                              num_kernels = 5,
                                                                              include_memory_kernels = True)

### Percentage of time spent by kernel type

In [5]:
kernel_type_metrics_df

,kernel_type,sum,percentage
0,COMMUNICATION,8040285,61.3
1,COMPUTATION,2671248,20.4
2,COMPUTATION overlapping COMMUNICATION,2119629,16.2
3,MEMORY,273227,2.1
4,COMMUNICATION overlapping MEMORY,16038,0.1
5,COMPUTATION overlapping MEMORY,564,0.0


### Statistics for each kernel

In [6]:
kernel_metrics_df

,name,sum (ns),max (ns),min (ns),stddev,mean (ns),kernel_type,rank
0,ncclKernel_AllGather_RING_LL_Sum_int8_t(ncclWo...,627683,10787,83,1651.592760,1687.319892,COMMUNICATION,0
1,ncclKernel_AllGather_RING_LL_Sum_int8_t(ncclWo...,644435,10884,82,1705.334758,1732.352151,COMMUNICATION,1
2,ncclKernel_AllGather_RING_LL_Sum_int8_t(ncclWo...,640631,10665,79,1700.774025,1722.126344,COMMUNICATION,2
3,ncclKernel_AllGather_RING_LL_Sum_int8_t(ncclWo...,643073,10834,81,1727.301230,1728.690860,COMMUNICATION,3
4,ncclKernel_AllGather_RING_LL_Sum_int8_t(ncclWo...,630605,10785,80,1656.166440,1695.174731,COMMUNICATION,4
...,...,...,...,...,...,...,...,...
107,Memset (Device),1134,13,1,0.807700,1.403465,MEMORY,3
108,Memset (Device),1084,8,1,0.739841,1.341584,MEMORY,4
109,Memset (Device),1073,14,1,0.818079,1.327970,MEMORY,5
110,Memset (Device),1038,7,1,0.693866,1.284653,MEMORY,6


## Kernel Attribution to Annotations

### Description

The PyTorch profile supports user marked annotations that can provide context on which module/component a kernel belongs to. This feature associates these user annoations to GPU kernels. The function returns a full dataframe of all gpu kernels; the user can perform aggregations on top of this.

### API

In [4]:
analyzer.get_gpu_kernels_with_user_annotations?

Signature:
analyzer.get_gpu_kernels_with_user_annotations(
    rank: int,
    expand_names: bool = True,
    shortern_names: bool = True,
) -> Optional[pandas.core.frame.DataFrame]
Docstring:
Provides a complete dataframe of GPU kernels and matches them to the corresponding user annotation i.e. user provided training phase. The output is a dataframe with all GPU kernel data alongside a "user_annotation" column.

Args:
    rank (int): Specify rank to return GPU kernels for.
    expand_names (bool): Expand integer name value to full names. This will add
        the columns "s_name" and "s_user_annotation" to the dataframe.
    shortern_names (bool): When expand_names is True, this flag enables shortening
        large CUDA kernel names. This works by removing the '<' template paramters etc.

Returns:
    pd.Dataframe:
        The returned dataframe has all trace columns along with "user_annotation",
        and optionally "s_user_annotation" column if expand_names=True.

Note: This API i

Note:
1. When there are multiple user annoations associated to a kernel, we currently attribute to the lowest annoation in the stack. A future improvement is to also suport a stack column.
1. Currently, the user must specify a rank to get annoatated GPU kernel data for.

### Example

In [ ]:
# This example uses a different trace that has user annotations

trace_dir_user_anno = hta_dir + "/tests/data/ns_resolution_trace/"
analyzer_user_anno = TraceAnalysis(trace_dir=trace_dir_user_anno)

In [24]:
gpu_kernels_df = analyzer_user_anno.get_gpu_kernels_with_user_annotations(rank=0, expand_names=True, shortern_names=False)

In [25]:
gpu_kernels_df[["s_name", "s_cat", "s_user_annotation", "stream", "ts", "dur"]]

,s_name,s_cat,s_user_annotation,stream,ts,dur
0,void at::native::(anonymous namespace)::CatArr...,kernel,DistributedDataParallel.forward,7,3488,5.0
1,void at::native::(anonymous namespace)::CatArr...,kernel,DistributedDataParallel.forward,7,3678,2.0
2,Memcpy DtoD (Device -> Device),gpu_memcpy,DistributedDataParallel.forward,7,4067,0.0
3,Memcpy DtoD (Device -> Device),gpu_memcpy,DistributedDataParallel.forward,7,4081,0.0
4,Memcpy DtoD (Device -> Device),gpu_memcpy,DistributedDataParallel.forward,7,4090,1.0
...,...,...,...,...,...,...
4871,void at::native::(anonymous namespace)::multi_...,kernel,void cudnn::cnn::reduce_wgrad_nchw_helper<floa...,7,387344,114.0
4872,void at::native::(anonymous namespace)::multi_...,kernel,void cudnn::cnn::reduce_wgrad_nchw_helper<floa...,7,387460,262.0
4873,void at::native::(anonymous namespace)::multi_...,kernel,void cudnn::cnn::reduce_wgrad_nchw_helper<floa...,7,387724,31.0
4874,void at::native::(anonymous namespace)::multi_...,kernel,void cudnn::cnn::reduce_wgrad_nchw_helper<floa...,7,387756,112.0


# Communication Computation Overlap

### Description
This feature computes the ratio of time spent communicating while computing and time spent communicating. Mathematically, it can be expressed as follows:

_time spent computing while communicating / time spent in communication_

### API

In [9]:
analyzer.get_comm_comp_overlap?

### Example

In [7]:
overlap_df = analyzer.get_comm_comp_overlap(visualize=False)

In [11]:
overlap_df

,rank,comp_comm_overlap_pctg
0,0,22.01
1,1,21.28
2,2,20.08
3,3,19.92
4,4,20.67
5,5,20.45
6,6,20.89
7,7,21.72


# Idle Time Breakdown

### Description
Understanding how much time a GPU is left idle and what are the causes can help direct optimization strategies. A GPU is considered idle when no kernel is running on it. Idle time is broken down into 3 categories

1. Host wait time: a GPU stream is idle because the CPU thread has not enqueued enough kernels to keep it occupied.
1. Kernel wait time: this is the short duration between kernels and is considered an overhead of launching multiple small kernels. We use a heuristic - if the kernel to kernel gap < a set threshold of 30ms - to determine if the gaps should be categorized as kernel-kernel wait.
1. Other wait time: in this case the idle time is attributed to an unknown cause. For example a compute kernel could be waiting for a CUDA event from a compute kernel.

### API

In [8]:
analyzer.get_idle_time_breakdown?

### Example

#### Calculate the idle time breakdown for all cuda streams on rank 0.

Note:
1. The `visualize_pctg (default=True)` option shows as bar chart per CUDA stream normalized to total idle time on the stream. You may choose to set this to False and see the absolute idle time breakdown on each stream.
1. The `show_idle_interval_stats (default=False)` option will compute stats of idle intervals between kernels. This should help get an idea of the distribution of idle intervals.

In [9]:
idle_time_df, interval_stats_df = analyzer.get_idle_time_breakdown(ranks=[1], visualize=False,\
                                                                   visualize_pctg = 1,
                                                                   show_idle_interval_stats=True)

2022-12-13 15:16:18,719 - hta - breakdown_analysis.py:L440 - INFO - Processing stream 7
2022-12-13 15:16:18,723 - hta - breakdown_analysis.py:L472 - INFO - Computing descriptive statistics for idle time intervals on stream 7:
2022-12-13 15:16:18,737 - hta - breakdown_analysis.py:L440 - INFO - Processing stream 20
2022-12-13 15:16:18,740 - hta - breakdown_analysis.py:L472 - INFO - Computing descriptive statistics for idle time intervals on stream 20:
2022-12-13 15:16:18,744 - hta - breakdown_analysis.py:L440 - INFO - Processing stream 26
2022-12-13 15:16:18,746 - hta - breakdown_analysis.py:L472 - INFO - Computing descriptive statistics for idle time intervals on stream 26:
2022-12-13 15:16:18,751 - hta - breakdown_analysis.py:L440 - INFO - Processing stream 24
2022-12-13 15:16:18,753 - hta - breakdown_analysis.py:L472 - INFO - Computing descriptive statistics for idle time intervals on stream 24:
2022-12-13 15:16:18,759 - hta - breakdown_analysis.py:L440 - INFO - Processing stream 28
2

In [10]:
idle_time_df

,rank,stream,idle_category,idle_time,idle_time_ratio
0,1,7,host_wait,942050.0,0.67
1,1,7,kernel_wait,36343.0,0.03
2,1,7,other,437053.0,0.31
3,1,20,host_wait,1694637.0,1.00
4,1,20,kernel_wait,10.0,0.00
5,1,20,other,0.0,0.00
6,1,26,host_wait,1057761.0,1.00
7,1,26,kernel_wait,2565.0,0.00
8,1,26,other,1794.0,0.00
9,1,24,host_wait,1254106.0,0.70


In [11]:
interval_stats_df

,rank,stream,count,mean,std,min,25%,50%,75%,max
idle_category,,,,,,,,,,
host_wait,1,7,2282.0,412.82,3252.98,8.0,37.25,79.0,166.00,111497.0
kernel_wait,1,7,6115.0,5.94,3.76,0.0,3.00,5.0,8.00,28.0
other,1,7,184.0,2375.29,2232.22,31.0,1166.75,1798.0,2912.50,14434.0
host_wait,1,20,33.0,51352.64,102824.64,174.0,195.00,2427.0,53074.00,364768.0
kernel_wait,1,20,2.0,5.00,2.83,3.0,4.00,5.0,6.00,7.0
other,1,20,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
host_wait,1,26,4.0,264440.25,176247.33,461.0,254460.50,347967.5,357947.25,361365.0
kernel_wait,1,26,337.0,7.61,7.43,1.0,3.00,4.0,8.00,28.0
other,1,26,30.0,59.80,20.08,30.0,58.00,59.0,59.00,141.0


# Augmented Counters (Queue length and Memory Bandwidth)

### Description
This feature adds counter/time series to the trace to visually debug time series.
One can select one or more of counters:
1. Queue Length: is defined as the number of outstanding CUDA operations on a stream.
    The queue depth is incremented on a kernel launch and decremented when a kernel executes.
1. Memory copy bandwidth: shows time series of memory bandwidth between a) host-device, b) device-host, c) device-device.

We can also get statistics for either one of queue length and memory copy bandwidth or both time series.

### API 1

In [95]:
analyzer.generate_trace_with_counters?

Signature:
analyzer.generate_trace_with_counters(
    time_series: Optional[hta.trace_analysis.TimeSeriesTypes] = None,
    ranks: Optional[List[int]] = None,
    output_suffix: str = '_with_counters',
) -> None
Docstring:
Adds a set of time series to the trace in order to aid debugging traces. Creates a new trace file
for each requested rank with the a suffix '_with_counters.json'. The following time series are
available in TimeSeriesTypes flag type.

1. Queue length - adds a time series to the trace indicating the size of the queue at any given time on each CUDA stream.
2. Memory copy bandwidth - adds a time series to the trace indicating the memory bandwidth used for device to host, host to device and device to device operations.

Either or both of the above can be enabled.

Args:
    time_series (Flag): Used to set the requested time series. Available values are
        TimeSeriesTypes.QUEUE_LENGTH and TimeSeriesTypes.MEMCPY_BANDWIDTH. By default
        both time series are added 

### Example 1

#### Generate a time series showing how Queues behave per CUDA stream on rank 0.

Note:
1. This call generates new trace file `rank-0_augmented.json.gz` at the trace_directory. One can open the trace file in Chrome Trace viewer or Perfetto.
1. Due to the way traces are trimmed during processing, the time series are not covering the last profiler step or iteration.

By default queue_length and memcpy bandwidth will be added. One can select the counters using the time_series argument that is a Flag field.
```
from hta.trace_analysis import TimeSeriesTypes
analyzer.generate_trace_with_counters(time_series = TimeSeriesTypes.QUEUE_LENTH)
```

In [99]:
from hta.trace_analysis import TimeSeriesTypes

analyzer.generate_trace_with_counters(ranks=[1], time_series = TimeSeriesTypes.QUEUE_LENGTH)

2024-08-22 11:29:45,339 - hta - trace_counters.py:L127 - INFO - Please note that the time series only contains points when the value changes. Once a values is observed the time series stays constant until the next update.
2024-08-22 11:29:47,197 - hta - trace_parser.py:L111 - WARNING - Parsed /Users/bcoutinho/Work/hta/queue_full_data/rank-1.Apr_10_22_19_54.3461.pt.trace.json.gz time = 1.63 seconds 
2024-08-22 11:29:47,201 - hta - trace_analysis.py:L335 - INFO - Writing trace with counters for rank 1 to /Users/bcoutinho/Work/hta/queue_full_data/rank-1.Apr_10_22_19_54.3461.pt.trace_with_counters.json.gz


### Example 2

#### Understanding statistics for queue_length on a CUDA stream for rank 0.

Statistics about the queue length can also be helpful to understand how well a CUDA stream is utilized.
The `get_queue_length_summary()` method returns a dataframe with statistics such as the mean, std, min, and 25, 50 and 75 percentiles per CUDA stream and rank.

### API 2

In [18]:
analyzer.get_queue_length_summary?

In [88]:
analyzer.get_queue_length_summary(ranks = [1])

2024-08-22 11:27:32,604 - hta - trace_counters.py:L127 - INFO - Please note that the time series only contains points when the value changes. Once a values is observed the time series stays constant until the next update.


queue_length                                                   \
                   count        mean         std  min   25%    50%    75%   
rank stream                                                                 
1    7           41296.0  366.792232  361.263264  0.0  28.0  252.0  692.0   
     21           1200.0    0.808333    1.064993  0.0   0.0    1.0    1.0   
     23            104.0    3.730769    2.213071  0.0   2.0    4.0    6.0   
     25            728.0   42.675824   26.017228  0.0  20.0   42.5   65.0   
     28              8.0    0.500000    0.534522  0.0   0.0    0.5    1.0   

                     
                max  
rank stream          
1    7       1024.0  
     21         7.0  
     23         8.0  
     25        88.0  
     28         1.0

### Raw Time Series
One can also investigate the time series directly using the function `get_queue_length_time_series()`.
This will return the actual time series of queue length for each rank.

In [111]:
analyzer.get_queue_length_time_series?

Signature: analyzer.get_queue_length_time_series(ranks: Optional[List[int]] = None) -> Dict[int, pandas.core.frame.DataFrame]
Docstring:
Queue length is defined as the number of outstanding CUDA operations on a stream. This
function calculates the time series for the queue length on each CUDA stream for the
specified ranks.

Args:
    ranks (List[int]): List of ranks for which the queue length time series is generated. Default = [0].

Returns:
    Dict[int, pd.DataFrame]
        Returns a dictionary whose key is the rank and value is a dataframe of queue length
        counter events. The following fields are in each row of the dataframe: ts (timestamp), pid (process id),
        tid (thread id), stream, and queue length.
File:      ~/Work/hta/HolisticTraceAnalysis2/hta/trace_analysis.py
Type:      method

In [112]:
queue_length_dict = analyzer.get_queue_length_time_series(ranks=[1])

2024-08-22 12:46:35,722 - hta - trace_counters.py:L127 - INFO - Please note that the time series only contains points when the value changes. Once a values is observed the time series stays constant until the next update.


### Example 3

#### Understanding time spent blocked on a kernel launch queue.

The GPU kernels launch queue is finite. If the CPU fills up this queue, the CPU will block till the GPU device launches kernels. We compute the time spent blocked on a full or max launch queue using `get_time_spent_blocked_on_full_queue()`

This function uses the queue length time series returned from `get_queue_length_time_series()` method above. The output is a dataframe with  two metrics : `duration_at_max_queue_length` and `relative_duration_at_max_queue_length` per `rank` and `stream`. Relative duration at max queue length considers the total duration of a trace and normalizes the `duration_at_max_queue_length`.

We can leverage this function to see if a job is blocked on launching too many GPU kernels. Max queue length used is a tunable parameter for this analysis.

### API 3

In [115]:
analyzer.get_time_spent_blocked_on_full_queue?

Signature:
analyzer.get_time_spent_blocked_on_full_queue(
    queue_length_dict: Dict[int, pandas.core.frame.DataFrame],
    max_queue_length: int = 1024,
) -> Optional[pandas.core.frame.DataFrame]
Docstring:
The GPU kernels launch queue is finite. If the CPU fills up this queue the CPU
will block till the GPU device launches kernels. We compute the time spent blocked
on a full launch queue in this function.

Returns an (optional) dataframe with the time spent on the kernel launch queue full.
This function takes the output from get_queue_length_time_series() and sums
up the time spent on all streams where the queue is full (see max_queue_length)

Args:
    queue_length_dict (Dict[int, pd.DataFrame]): A dictionary of rank -> time series with the queue length of each CUDA stream.
        This is the output of get_queue_length_time_series().
    max_queue_length (int): Max kernel launch queue length.

Returns:
    Optional[pd.DataFrame]
        An (optional) dataframe containing the summa

In [116]:
analyzer.get_time_spent_blocked_on_full_queue(queue_length_dict, max_queue_length=1024)

2024-08-22 12:52:45,028 - hta - trace_counters.py:L237 - INFO - Rank=1, stream=7, total dur at max_queue = 183385.0


,rank,stream,duration_at_max_queue_length,relative_duration_at_max_queue_length
0,1,7,183385.0,0.088301


# CUDA Kernel Launch Statistics

### Description
For each event launched on the GPU there is a corresponding scheduling event on the CPU. These events are linked by a common correlation id. This feature computes the duration of the CPU operator, its corresponding GPU kernel and the launch delay i.e. the difference between GPU kernel starting and CPU operator ending.

The plots generated provide distribution of GPU kernels which have very small/large duration and events which have excessive launch delay. The runtime_cutoff and launch_delay_cutoff arguments in API allow the user to tune the definition of an outlier.

### API

In [22]:
analyzer.get_cuda_kernel_launch_stats?

### Example

#### Calculate the duration of each GPU kernel, its corresponding CUDA Launch Kernel event and the launch delay.

In [13]:
kernel_stats = analyzer.get_cuda_kernel_launch_stats(visualize=False)

In [14]:
kernel_stats[0]

,correlation,cpu_duration,gpu_duration,launch_delay
0,278204,15,37,31
1,278209,12,40,26
2,278239,12,31,25
3,278244,11,53,24
4,278249,11,3,25
...,...,...,...,...
10511,335216,13,103,24418
10512,335221,10,50,24471
10513,335229,12,55,24451
10514,335252,16,3045,21331


# Most Frequent CUDA Kernel Sequences

### Description

This feature finds the sequence of most frequent CUDA kernels launched for any specified operator. It generates a new trace file which overlays the top-k identified patterns on the original trace file. Searching for the keyword __"Patterns"__ in the new trace file highlights the relevant CPU and GPU ops. The highlighted events indicate where to look for opportunities to fuse CUDA kernels or CPU ops.

### API

In [25]:
analyzer.get_frequent_cuda_kernel_sequences?

### Example

#### Calculate the top 5 most frequent sequences of CUDA kernels of length 3 launched for the `aten::linear` operator on rank 0.

Note:
1. Before executing this function, change the `output_dir` value to an existing folder on your local machine.
1. Load the generated trace file in Chrome Tracing and search for the keyword “Pattern”. The lower panel will enumerate the kernel names and their frequency. The kernel names are separated by the pipe symbol “|”. 
1. The log messages will contain the output path of the generated trace file.

In [ ]:
frequent_cuda_kernels = analyzer.get_frequent_cuda_kernel_sequences(
                            operator_name="aten::linear", 
                            output_dir="/tmp/",
                            min_pattern_len = 3,
                            rank = 0,
                            top_k = 5,
                            visualize=False)

In [16]:
frequent_cuda_kernels

,pattern,count,GPU kernel duration (ns),CPU op duration (ns)
0,aten::linear|void at::native::unrolled_element...,96,26319,25268
1,aten::linear|void at::native::unrolled_element...,68,34546,17078
2,aten::linear|Memset (Device)|volta_fp16_s884ge...,48,11300,9652
3,aten::linear|void at::native::unrolled_element...,4,537,1084
